# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook.

In [1]:
from pprint import pprint
from pathlib import Path

import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import load_yaml

pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True, until=10000)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience only
metrics = sim.metrics

## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm")
print(f"Project total: {total * 100:.1f}%")

Project total: 97.2%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.98,0.97,0.99,0.99,0.98,0.82,0.99,0.98,0.99,1.00,0.97,0.98,1.00,0.97,0.98,1.00,0.98,0.99,0.98,0.99,1.00,0.72,0.99,0.98,1.00,0.99,1.00,0.99,0.98,0.99,1.00,0.98,0.80,1.00,0.98,0.97,0.99,0.98,0.99,0.98,1.00,0.98,0.98,1.00,0.98,0.98,0.99,1.00,0.98,0.99,0.98,0.99,0.99,0.99,0.98,0.97,0.98,1.00,0.98,0.98,0.98,0.99,1.00,0.98,0.98,0.99,1.00,0.91,1.00,0.75,0.70,0.99,0.99,0.98,0.99,0.98,0.96,1.00,1.00,0.99


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.97
2004,0.95


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.97
2,0.97
3,0.97
4,0.97
5,0.97
6,0.98
7,0.99
8,0.98
9,0.99


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.99
     2          0.98
     3          0.97
     4          0.97
     5          0.97
     6          0.98
     7          0.99
     8          0.98
     9          0.99
     10         0.98
     11         0.95
     12         0.95
2004 1          0.95
     2          0.95

## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

**Question: Are AEP calculations and losses computed for the capacity factor?** Might want to specify either way.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm")
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm")
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.43%
Gross Capacity Factor: 0.44%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.43,0.42,0.43,0.43,0.43,0.37,0.43,0.43,0.43,0.44,0.43,0.43,0.44,0.42,0.43,0.44,0.43,0.44,0.43,0.44,0.44,0.28,0.44,0.43,0.44,0.44,0.44,0.43,0.43,0.44,0.44,0.43,0.35,0.44,0.43,0.42,0.44,0.43,0.44,0.43,0.44,0.43,0.42,0.44,0.43,0.43,0.43,0.44,0.43,0.44,0.43,0.44,0.43,0.44,0.43,0.43,0.43,0.44,0.43,0.43,0.43,0.43,0.44,0.43,0.43,0.43,0.44,0.41,0.44,0.33,0.28,0.44,0.43,0.43,0.44,0.43,0.43,0.44,0.44,0.44


In [10]:
# BUG: I cannot make this cell execute.
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.54


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.58
2,0.45
3,0.37
4,0.52
5,0.34
6,0.31
7,0.30
8,0.30
9,0.28


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm").head(24)

windfarm
year month          
2003 1          0.63
     2          0.36
     3          0.37
     4          0.52
     5          0.34
     6          0.31
     7          0.30
     8          0.30
     9          0.28
     10         0.43
     11         0.53
     12         0.54
2004 1          0.54
     2          0.56

## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project")
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project")
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project")
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 54%
Unscheduled Task Completion Rate: 99%
    Overall Task Completion Rate: 96%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

ValueError: Must pass 2-d input. shape=(2, 977, 1)

In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

ValueError: Must pass 2-d input. shape=(2, 977, 1)

In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year").head(24)

ValueError: Must pass 2-d input. shape=(2, 977, 1)

## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $48,513.18/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"729,367.19","729,367.19","729,367.19","266,000.00","9,189,062.50"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"11,372,843.75"
2004,"270,320.31"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"109,010.42","109,010.42","109,010.42",0.00,0.00
2,"84,856.77","84,856.77","84,856.77",0.00,0.00
3,"54,250.00","54,250.00","54,250.00",0.00,0.00
4,"52,500.00","52,500.00","52,500.00",0.00,0.00
5,"54,250.00","54,250.00","54,250.00",0.00,"3,689,062.50"
6,"52,500.00","52,500.00","52,500.00","266,000.00",0.00
7,"54,250.00","54,250.00","54,250.00",0.00,"2,750,000.00"
8,"54,250.00","54,250.00","54,250.00",0.00,0.00
9,"52,500.00","52,500.00","52,500.00",0.00,"2,750,000.00"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True).head(24)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
     6                   52,500.00               52,500.00   
     7                   54,250.00               54,250.00   
     8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2004 1                   54,250.00               54,250.00   
     2                   35,856.77               35,856.77   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00       3,689,062.50  
     6                   52,500.00            266,000.00               0.00  
     7                   54,250.00                  0.00       2,750,000.00  
     8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00       2,750,000.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00               0.00  
2004 1                   54,250.00                  0.00               0.00  
     2                   35,856.77                  0.00               0.00

## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,0.71,0.68,0.66,0.17,0.61


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,0.68,0.65,0.64,0.17,0.62
2004,0.90,0.90,0.87,0.00,0.00


## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

**Question: All these labor calculations are $0.** Is this intended?

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [24]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [25]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [26]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00


In [27]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [28]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False).head(24)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
     6                  0.00
     7                  0.00
     8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2004 1                  0.00
     2                  0.00

## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [29]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"165,557.29"
Repair,"2,293,458.33"
Crew Transfer,"162,106.77"
Site Travel,0.00
Mobilization,"1,500,000.00"
Weather Delay,"1,775,427.08"
No Requests,"2,884,065.10"
Not in Shift,"2,862,549.48"


In [30]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"165,557.29","165,557.29"
Repair,0.00,0.00,0.00,"2,293,458.33","2,293,458.33"
Crew Transfer,0.00,0.00,0.00,"162,106.77","162,106.77"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"1,500,000.00","1,500,000.00"
Weather Delay,0.00,0.00,0.00,"1,775,427.08","1,775,427.08"
No Requests,0.00,0.00,0.00,"2,884,065.10","2,884,065.10"
Not in Shift,0.00,0.00,0.00,"2,862,549.48","2,862,549.48"


In [31]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False).head(10)

total_cost
year reason                    
2003 Maintenance     100,515.63
     Repair        2,259,114.58
     Crew Transfer   153,812.50
     Site Travel           0.00
     Mobilization  1,500,000.00
     Weather Delay 1,734,739.58
     No Requests   2,884,065.10
     Not in Shift  2,740,596.35
2004 Maintenance      65,041.67
     Repair           34,343.75

In [32]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False).head(10)

total_cost
month reason                   
1     Maintenance     37,388.02
      Repair          52,736.98
      Crew Transfer    8,476.56
      Site Travel          0.00
      Mobilization         0.00
      Weather Delay   49,729.17
      No Requests     82,869.79
      Not in Shift    95,830.73
2     Maintenance     27,653.65
      Repair          37,606.77

In [33]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False).head(20)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          32,210.94
           Crew Transfer    3,554.69
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   22,166.67
           No Requests     82,869.79
           Not in Shift    23,479.17
     2     Maintenance          0.00
           Repair          23,789.06
           Crew Transfer    2,424.48
           Site Travel          0.00
           Mobilization         0.00
           Weather Delay   11,739.58
           No Requests     97,507.81
           Not in Shift    11,539.06
     3     Maintenance          0.00
           Repair          22,221.35
           Crew Transfer    2,442.71
           Site Travel          0.00

## Component

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [34]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
generator,"10,958,942.71"


In [35]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
generator delay                  0.00              0.00    4,626,820.31   
          maintenance            0.00              0.00      165,557.29   
          repair                 0.00              0.00    2,293,458.33   

                        total_cost  
component action                    
generator delay       4,626,820.31  
          maintenance   165,557.29  
          repair      2,293,458.33

In [36]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,generator,"3,364,500.00",0.00,"6,978,497.40","10,342,997.40"
2004,generator,"346,500.00",0.00,"269,445.31","615,945.31"


In [37]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,generator,"319,500.00",0.00,"240,880.21","560,380.21"
2,generator,"202,500.00",0.00,"155,312.50","357,812.50"
3,generator,"85,000.00",0.00,"35,783.85","120,783.85"
4,generator,"93,000.00",0.00,"41,398.44","134,398.44"
5,generator,"1,062,500.00",0.00,"3,231,700.52","4,294,200.52"
6,generator,"222,500.00",0.00,"83,958.33","306,458.33"
7,generator,"702,500.00",0.00,"2,004,187.50","2,706,687.50"
8,generator,"40,500.00",0.00,"46,393.23","86,893.23"
9,generator,"426,500.00",0.00,"1,013,046.88","1,439,546.88"


In [38]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     generator      115,500.00              0.00       78,130.21   
     2     generator       60,000.00              0.00       48,617.19   
     3     generator       85,000.00              0.00       35,783.85   
     4     generator       93,000.00              0.00       41,398.44   
     5     generator    1,062,500.00              0.00    3,231,700.52   
     6     generator      222,500.00              0.00       83,958.33   
     7     generator      702,500.00              0.00    2,004,187.50   
     8     generator       40,500.00              0.00       46,393.23   
     9     generator      426,500.00              0.00    1,013,046.88   
     10    generator       90,000.00              0.00       98,674.48   
     11    generator      264,500.00              0.00      155,312.50   
     12    generator      202,000.00              0.00      141,294.27   
2004 1     generator      204,000.00              0.00      162,750.00   
     2     generator      142,500.00              0.00      106,695.31   
     3     generator            0.00              0.00            0.00   
     4     generator            0.00              0.00            0.00   
     5     generator            0.00              0.00            0.00   
     6     generator            0.00              0.00            0.00   
     7     generator            0.00              0.00            0.00   
     8     generator            0.00              0.00            0.00   
     9     generator            0.00              0.00            0.00   
     10    generator            0.00              0.00            0.00   
     11    generator            0.00              0.00            0.00   
     12    generator            0.00              0.00            0.00   

                       total_cost  
year month component               
2003 1     generator   193,630.21  
     2     generator   108,617.19  
     3     generator   120,783.85  
     4     generator   134,398.44  
     5     generator 4,294,200.52  
     6     generator   306,458.33  
     7     generator 2,706,687.50  
     8     generator    86,893.23  
     9     generator 1,439,546.88  
     10    generator   188,674.48  
     11    generator   419,812.50  
     12    generator   343,294.27  
2004 1     generator   366,750.00  
     2     generator   249,195.31  
     3     generator         0.00  
     4     generator         0.00  
     5     generator         0.00  
     6     generator         0.00  
     7     generator         0.00  
     8     generator         0.00  
     9     generator         0.00  
     10    generator         0.00  
     11    generator         0.00  
     12    generator         0.00

## Fixed Cost Impacts

**Question: Are some of these costs supposed to be 0?** Might want to explain.

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [39]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [40]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"1,826,493.15"


In [41]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"1,826,493.15"


In [42]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,operations_all_risk,business_interruption,third_party_liability,storm_coverage,submerge_land_lease_costs,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"1,826,493.15"


In [43]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"226,485.15"


## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [44]:
# Project totals at the project level
total = metrics.process_times()
total

/Users/akey/Projects/WOMBAT/WOMBAT/wombat/core/post_processor.py:1310: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(


,time_to_completion,process_time,downtime,N
category,,,,
annual service,"17,097.50","5,468.00","4,737.50",67.00
major repair,"4,486.44",106.00,63.00,5.00
major replacement,"9,179.11",980.25,"9,179.11",8.00
manual reset,"19,164.55","3,751.75","1,687.00",600.00
medium repair,"2,082.83","1,653.00","1,327.25",34.00
minor repair,"8,490.09","3,930.75","2,061.00",263.00


## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [45]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

/Users/akey/Projects/WOMBAT/WOMBAT/wombat/core/post_processor.py:1372: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  production = self.production.sum(axis=0)[col_filter]


,windfarm
Project Energy Production (kWh),"1,021,389,024.50"


In [46]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

/Users/akey/Projects/WOMBAT/WOMBAT/wombat/core/post_processor.py:1372: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  production = self.production.sum(axis=0)[col_filter]


,windfarm,S00T1,S00T2,S00T3,S00T4,S00T5,S00T6,S00T7,S00T8,S00T9,S00T10,S00T11,S00T12,S00T13,S00T14,S00T15,S00T16,S00T17,S00T18,S00T19,S00T20,S00T21,S00T22,S00T23,S00T24,S00T25,S00T26,S00T27,S00T28,S00T29,S00T30,S00T31,S00T32,S00T33,S00T34,S00T35,S00T36,S00T37,S00T38,S00T39,S00T40,S00T41,S00T42,S00T43,S00T44,S00T45,S00T46,S00T47,S00T48,S00T49,S00T50,S00T51,S00T52,S00T53,S00T54,S00T55,S00T56,S00T57,S00T58,S00T59,S00T60,S00T61,S00T62,S00T63,S00T64,S00T65,S00T66,S00T67,S00T68,S00T69,S00T70,S00T71,S00T72,S00T73,S00T74,S00T75,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"1,021,389,024.50","13,009,594.50","12,655,912.00","13,040,439.00","12,998,828.50","12,906,724.00","11,072,807.00","12,911,457.50","12,929,291.00","12,794,917.50","13,135,541.50","12,787,059.00","12,894,490.00","13,142,413.00","12,687,631.50","12,966,495.50","13,129,559.00","12,869,808.50","13,160,183.50","12,834,542.00","13,111,836.50","13,131,909.00","8,506,805.50","13,056,046.50","12,864,734.00","13,098,095.00","13,087,620.50","13,147,418.00","12,943,538.00","12,877,653.50","13,078,966.00","13,155,938.00","12,963,953.50","10,494,070.00","13,169,553.00","12,832,692.50","12,655,725.00","13,092,270.50","12,972,486.00","13,108,844.00","12,818,868.00","13,128,320.50","12,863,114.50","12,731,591.00","13,087,389.50","12,888,704.00","12,996,500.00","13,008,640.00","13,086,678.50","12,897,091.50","13,120,610.50","12,938,542.00","13,067,247.50","13,022,575.50","13,069,377.50","12,861,344.50","12,794,372.50","12,786,771.50","13,083,772.50","12,821,412.50","12,950,667.50","13,004,450.50","12,818,491.00","13,147,790.50","12,963,139.50","12,967,606.50","12,932,572.00","13,162,029.50","12,374,189.50","13,146,937.50","9,915,429.50","8,254,403.50","13,135,368.00","13,002,785.50","12,971,827.00","13,080,957.00","13,046,142.50","12,807,886.00","13,120,161.50","13,087,748.00","13,147,668.50"


In [47]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"859,235,285.50"
2004,"162,153,739.00"


In [48]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"207,501,317.50"
2,"125,215,094.00"
3,"66,039,142.00"
4,"90,443,606.00"
5,"60,602,193.00"
6,"53,217,800.50"
7,"53,431,887.50"
8,"52,991,939.00"
9,"47,656,662.50"


In [49]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False).head(24)

windfarm
year month               
2003 1     111,965,498.50
     2      58,597,174.00
     3      66,039,142.00
     4      90,443,606.00
     5      60,602,193.00
     6      53,217,800.50
     7      53,431,887.50
     8      52,991,939.00
     9      47,656,662.50
     10     76,725,627.50
     11     91,736,896.00
     12     95,826,859.00
2004 1      95,535,819.00
     2      66,617,920.00

## PySAM-Powered Results

The PySAM library is utilized for a number of project financial metrics.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [50]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [51]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [ ]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

### After-tax Internal Return Rate (IRR)

In [ ]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [52]:
SAM_settings = "SAM_Singleowner_defaults.yaml"
metrics.sam_settings = load_yaml(sim.env.data_dir / "windfarm", SAM_settings)
metrics._setup_pysam()

Exception: singleowner execution error.
	exec fail(singleowner): PPA price from which to calculate parasitic load costs is not specified. Check inputs for Revenue and Electricity Purchases.



In [53]:
metrics.pysam_all_outputs()

Exception: Runtime error: get_cf_project_return_aftertax_npv called for SAM_Singleowner but "cf_project_return_aftertax_npv" not assigned


In [54]:
sim.env.cleanup_log_files(log_only=False)